In [16]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
  

In [ ]:
testing_labels_final = np.array(testing_labels)
training_labels_final = np.array(training_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='pre'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [34]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(padded[3])
print(len(padded[3]))
print(training_sentences[3])
print(len(training_sentences[3].split()))

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  360    7    2  239    5   20   16    4 8837 2705 2679   55
    2  367    5    2  179   58  141 1419   17   94  203  980   15   23
    1   86    4  193 3134 3069    3    1   16    4  383    5  640  395
  361   38 5863    3 5741 1859   15  208 8659 3471    2  115  376   44
   25   61    1    6 1681   61 1846 4127   43    4 2289   

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5646 - accuracy: 0.7192 - val_loss: 0.4405 - val_accuracy: 0.8440
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3389 - accuracy: 0.8960 - val_loss: 0.4235 - val_accuracy: 0.8303
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1863 - accuracy: 0.9607 - val_loss: 0.4427 - val_accuracy: 0.8288
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1114 - accuracy: 0.9816 - val_loss: 0.5049 - val_accuracy: 0.8215
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0802 - accuracy: 0.9868 - val_loss: 0.5538 - val_accuracy: 0.8204
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0654 - accuracy: 0.9886 - val_loss: 0.5549 - val_accuracy: 0.8250
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0578 - accuracy: 0.9892 - val_loss: 0.5943 - val_accuracy: 0.8222
Epoch 

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
f=model.layers[3]
weights2=f.get_weights()[0]
print(weights2.shape)
print(weights.shape) # shape: (vocab_size, embedding_dim)

(6, 1)
(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

After this go to http://projector.tensorflow.org/ to project the embeddings with their word labels in the 3D space. Use vecs.tsv and meta.csv file. 
This helps give a visual representation of the positive and negative words.